In [43]:
import pandas as pd
import numpy as np

# Leer tabla RAW
raw_path = '../Raws/RAW_24-03-2024.csv'
raw_data = pd.read_csv(raw_path) 

C:\Users\Javier\AppData\Local\Temp\ipykernel_12880\1841553572.py:6: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(raw_path)


In [88]:
raw_emotions = raw_data.iloc[:, :3].join(raw_data.iloc[:, 53:73])

# Separar la RAW por takes
emotions_takes = {}

for (subjectID, take), emotion_take in raw_emotions.groupby(['Subject ID', 'Take']):
    take_name = f'Subject {subjectID} Take {take}'
    take_recordings = {}
    
    # Para cada grabacion de emociones
    for i in range(3):
        # Si no hay grabacion de emociones, paramos
        dropped_emotions = emotion_take.iloc[:, (3 + i * 5): (8 + i * 5)].dropna()
        
        if dropped_emotions.size == 0:
            #print(f"No hay grabacion numero {i + 1} de {take_name}")
            break

        recording_name = f'{take_name} Recording {i + 1}'
        take_recordings[recording_name] = emotion_take.iloc[:, (3 + i * 5): (8 + i * 5)]

        # Empezar con 3 en el instante 0, y 1 en inmersion
        take_recordings[recording_name].iloc[0, :] = 3
        take_recordings[recording_name].iloc[0, 4] = 1

        # Rellenar NaN replicando la ultima entrada hasta que se encuentre una nueva
        take_recordings[recording_name].ffill(inplace=True)

        #print(f'{take_name} Recording {i + 1}')
        #display(take_recordings[recording_name])

    # Realizar la media de los TableApps disponibles
    emotions_takes[take_name] = emotion_take.iloc[:, :3]
    
    for i, emotion_name in zip(range(5), ['Focus', 'Amusement', 'Surprise', 'Fright', 'Inmersion']):
        emotion_values = [take_recordings[recording_name].iloc[:, i] for recording_name in take_recordings]
        emotion_values_df = pd.concat(emotion_values, axis=1, ignore_index=True)

        emotion_mean = pd.DataFrame(emotion_values_df.mean(axis=1))
        emotion_mean.columns = [emotion_name]

        emotions_takes[take_name] = pd.concat([emotions_takes[take_name], emotion_mean], axis=1)

    #display(emotions_takes[take_name])

# Concatenar de nuevo todos los takes
preprocessed_emotions = pd.concat(emotions_takes.values(), ignore_index=True)

# Reescalar emociones
columnas_a_reescalar = preprocessed_emotions.columns[3:]

for columna in columnas_a_reescalar:
    preprocessed_emotions[columna] = (preprocessed_emotions[columna] - 3) / (5 - 3)

display(preprocessed_emotions)
    

,time,Subject ID,Take,Focus,Amusement,Surprise,Fright,Inmersion
0,0.000,1,1,3.0,3.0,3.0,3.0,1.0
1,0.004,1,1,3.0,3.0,3.0,3.0,1.0
2,0.008,1,1,3.0,3.0,3.0,3.0,1.0
3,0.012,1,1,3.0,3.0,3.0,3.0,1.0
4,0.016,1,1,3.0,3.0,3.0,3.0,1.0
...,...,...,...,...,...,...,...,...
1410675,615.908,18,1,4.5,3.0,3.0,2.5,5.0
1410676,615.912,18,1,4.5,3.0,3.0,2.5,5.0
1410677,615.916,18,1,4.5,3.0,3.0,2.5,5.0
1410678,615.920,18,1,4.5,3.0,3.0,2.5,5.0


,time,Subject ID,Take,Focus,Amusement,Surprise,Fright,Inmersion
0,0.000,1,1,0.00,0.0,0.0,0.00,-1.0
1,0.004,1,1,0.00,0.0,0.0,0.00,-1.0
2,0.008,1,1,0.00,0.0,0.0,0.00,-1.0
3,0.012,1,1,0.00,0.0,0.0,0.00,-1.0
4,0.016,1,1,0.00,0.0,0.0,0.00,-1.0
...,...,...,...,...,...,...,...,...
1410675,615.908,18,1,0.75,0.0,0.0,-0.25,1.0
1410676,615.912,18,1,0.75,0.0,0.0,-0.25,1.0
1410677,615.916,18,1,0.75,0.0,0.0,-0.25,1.0
1410678,615.920,18,1,0.75,0.0,0.0,-0.25,1.0
